# maude-magic

> Execute a maude`s session in Jupyter Lab

**Refs:**
- [pexpect](https://pexpect.readthedocs.io/en/stable/)
- [nbdev](https://nbdev.fast.ai/tutorials/tutorial.html)
- [IPython magics](https://ipython.readthedocs.io/en/stable/config/custommagics.html)
- See persistent Shells.ipynb  

In [ ]:
%%bash 
for package in pexpect nbdev jupyterlab-quarto
do  
    pip list | grep   "^$package *" > /dev/null && echo "$package found" || pip install $package
done
# This must be done in terminal
# nbdev_install_quarto

## The Maude Interpreter class

In [ ]:
#| default_exp maude-magic

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os
import pexpect
import time

In [ ]:
#| export
timeout = 3

class TimeoutException(Exception):
    """Exception raised if time-out."""

    def __init__(self):
        super().__init__("Timeout exception.")
        self.error_code = 1

    def __str__(self):
        return f"{self.message} (Error Code: {self.error_code})"    


class MaudeInterpreter:
    """Controls maude execution, executing commands and print responses.
       Preserve sessions between different cell executions."""
    def __init__(self,debug=False,timeout=timeout):
        """ Init variables and spawns maude. """
        self.debug,self.timeout = debug,timeout
        # Execute maude in environment "env". Add current working directory to MAUDE_LIB
        env = dict(os.environ)
        if not 'MAUDE_LIB' in env: raise(Exception('MAUDE_LIB environmet variable not found'))
        env['MAUDE_LIB'] += ':' + os.getcwd()
        if self.debug: 
            print(f"MAUDE_LIB={env['MAUDE_LIB']}")
        # the expawned process:
        self.sh = pexpect.spawn('maude', encoding='utf-8', env=env)
        self.sh.expect('Maude> ')
        self._sync()    

    def _sync(self):
        """ We syncrhonize with maude shell requesting a pwd.""" 
        self.sh.sendline('pwd')
        self.sh.expect('pwd.*Maude> ')
            
    def __del__(self):
        if self.debug: print('Destroying Object')
        self.__call__('quit .')
             
    # Cmmand processing
    def __call__(self,command,timeout=timeout):
        """ Adds '\n' to command if it don't terminate with it. """
        # Don't terminate session by command
        if self.debug: print(f"Original Command = {repr(command)}")
        if command == 'quit .': return ''
        # strip command before send
        if self.debug: print(f"<--{repr(command.strip())}")
        if command[-1] != '\n' : command += '\n'
        if self.debug: print(f"Sent Command = {repr(command)}")
        self.sh.send(command)
        self._sync()
        if self.debug: print(f"-->{repr(self.sh.before)+repr(self.sh.after)}")
        response = self.sh.before
        # filter response
        return response
        

Creating maude interpreter:

In [ ]:
maude=MaudeInterpreter(debug=True)

Load a maude module. Show it and make some reduction:

In [ ]:
print(maude('load SIMPLE-NAT .'))
print(maude('show module .'))
print(maude('red s s zero .'))

Skip `quit` command:

In [ ]:
maude('quit .')

Closing maude session on object destroy:

In [ ]:
del maude 

## The maude Magic Class

In [ ]:
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

In [ ]:
# This code can be put in any Python module, it does not require IPython
# itself to be running already.  It only creates the magics subclass but
# doesn't instantiate it yet.
# from __future__ import print_function
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MaudeMagics(Magics):
    """Adapts Maude Shell to a IPython Magic class.Uses a owned Maude Shell.
       Cell magics are used to execute maude commands.
       Line magics are used for command line options."""  
    def __init__(self,shell):
        # Create the owned Maude Shell instance an pass it as shell
        super(MaudeMagics,self).__init__(MaudeInterpreter(debug=False)) 
        # print("On Construntor:"+str(type(self.shell)))
        self.line_counter=0
        
    def prepare_request(self,cell_contents:str)->str:
        """ - Removes leading and trailing empty lines from cell
            - Removes leading and trailing empty lines from each line
            - Terminates each line with \r\n
            - Count efective lines sent to maude shell.
        """
        # print(f"cell_contents at maude()={cell_contents}")
        cell_contents=cell_contents.strip()
        request = ""
        for cell_line in cell_contents.split('\n'):
            request+=cell_line.strip()+'\n'
            self.line_counter+=1
        # Maude shell will add trailing \n
        return request
    
    __prepare_request=prepare_request 

    def prepare_response(self,shell_response:str)->str:
        """ Ads the count of sent lines at the header of respone,
            to ease sintax error location.
        """    
        response =  f"{self.line_counter} (lines sent before.)\n"
        response += ("--------------------------------------------\n")
        response += shell_response + '\n'
        return response
    
    __prepare_response = prepare_response     
           
    @line_cell_magic
    def maude(self, line, cell=None):
        if cell is None:
            return self.shell(line)
        else:
            # print(f"cell at maude()={cell}")
            print(self.prepare_response(self.shell(self.prepare_request(cell))))

# In order to actually use these magics, you must register them with a
# running IPython.

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(MaudeMagics)

Manually executing 'load_ipython_extension' for test purposes:

In [ ]:
load_ipython_extension(get_ipython())

Now, MaudeMagic uses an owned MaudeInterpreter to run maude commands:

In [ ]:
result = %maude load SIMPLE-NAT . 
result
    #assert result == 'Maude>'

In [ ]:
%%maude
show module .  

In [ ]:
%%maude
red s s zero .    

## Use cases as test

### JSON Syntax

[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/)

In [ ]:
%%maude

fmod JSON is
    ------------------------
    sort Json .
    -----------------------    
    --- Section 3.4 Defines JSON Objects 
    protecting STRING * (op + : String String -> String to & ) .
    protecting FLOAT . 
    --- For JSON syntax we use Number, String, Boolean, null, Array and JSON    
    --- subsorts Rat Bool < Json .    
    subsort  String < Json .
        
    sorts Json+ Json* .
    subsorts Json < Json+ < Json* .    

    op nil-Json* : -> Json* [ctor] .        
    op _,_ : Json* Json*   -> Json* [ctor id: nil-Json*] .
    op _,_ : Json* Json+   -> Json+ [ditto] .        
    op _,_ : Json+ Json*   -> Json+ [ditto] .            
    op _,_ : Json+ Json+   -> Json+ [ditto] .                
    

        
    sorts  Pair Pair+ Pair* .

    op _:_ : String Json -> Pair [ctor prec 30] .
    
    subsorts Pair < Pair+ < Pair* .
           
    op nil-Pair* : -> Pair* [ctor] .
   
    op _,_ : Pair* Pair*   -> Pair* [ctor assoc comm id: nil-Pair*] .
    op _,_ : Pair* Pair+   -> Pair+ [ditto] .
    op {_} : Pair*         -> Json  [ctor] . 
    op [_] : Json*         -> Json  [ctor] . 
        
endfm

        

In [ ]:
%%maude
red "Juan" : "Perico" .

In [ ]:
%%maude

red 
{
  "name" : "Manu Sporny" ,
  "homepage" : "http://manu.sporny.org/" ,
  "image" : "http://manu.sporny.org/images/manu.png"
} .

In [ ]:
%%maude 
parse "Juan" , "Perico" .


In [ ]:
%%maude
parse "Juan",{ "id" : "perico" } .

In [ ]:
%%maude 
parse [ "A", "B"] .

In [ ]:
%%maude 
parse --- Example 2: Sample JSON document
{
  "name" : "Manu Sporny",
  "homepage" : "http://manu.sporny.org/",
  "image" : "http://manu.sporny.org/images/manu.png"
} .

In [ ]:
%%maude 
--- Example 4
red 
{
  "@context" : {
    "name" : "http://schema.org/name",
    "image" : {
      "@id" : "http://schema.org/image",
      "@type" : "@id"
    },
    "homepage" : {
      "@id" : "http://schema.org/url",

        "@type" : "@id"
    }
  }
} .

#### IRIS
[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/#terms)

In [89]:
%%maude
fmod IRIS is
protecting JSON .

sort IRI .
subsort IRI < String .
cmb S:String : IRI if find(S:String,":",0) :: NzNat .
endfm    


205 (lines sent before.)
--------------------------------------------




In [90]:
result = %maude red "Http://perico" .
assert 'IRI: "Http://perico"'   

#### Keywords

In [99]:
%%maude
fmod KEYWORD is
    protecting STRING .
    sort Keyword .
    subsort Keyword < String .
    mb "@context"  : Keyword .
    mb "@id"       : Keyword .
    mb "@context"  : Keyword .
    mb "@id"       : Keyword .
    mb "@included" : Keyword .
    mb "@graph"    : Keyword .
    mb "@nest"     : Keyword .
    mb "@type"     : Keyword .
    mb "@reverse"  : Keyword .
    mb "@index"    : Keyword .
endfm    


271 (lines sent before.)
--------------------------------------------
fmod KEYWORD is
> protecting STRING .
> sort Keyword .
> subsort Keyword < String .
> mb "@context"  : Keyword .
> mb "@id"       : Keyword .
> mb "@context"  : Keyword .
> mb "@id"       : Keyword .
> mb "@included" : Keyword .
> mb "@graph"    : Keyword .
> mb "@nest"     : Keyword .
> mb "@type"     : Keyword .
> mb "@reverse"  : Keyword .
> mb "@index"    : Keyword .
> endfm
Advisory: redefining module KEYWORD.
Maude> 



In [100]:
result = %maude red "@context" . 
assert 'Keyword: "@context"' in result     

#### Terms

In [102]:
%%maude
fmod TERM is
    sort Term .
    subsort Term < String .
    cmb S:String : Term  if find(S:String,":",0) = notFound 
                         /\ not substr(S:String,0,1) ="@" .
endfm

283 (lines sent before.)
--------------------------------------------
fmod TERM is
> sort Term .
> subsort Term < String .
> cmb S:String : Term  if find(S:String,":",0) = notFound
> /\ not substr(S:String,0,1) ="@" :
> endfm
> 



### JSON-LD BASIC

In [ ]:
%%maude
fmod JSON-LD-BASIC is
    protecting JSON * (sort Pair to Entry, Pair+ to Entry+, Pair* to Entry*,
                       sort Json to Value) .

    sorts Term IRI Keyword .
    subsorts Term IRI Keyword < String .

    vars S : String . 
        
    cmb S : Keyword if substr(S,0,1) == "@" .
    cmb S : IRI     if find(S,":",0) =/= notFound .
    cmb S : Term    if find(S,":",0) == notFound /\ substr(S,0,1) =/= "@" .    

    --- The Context 
 
    sort Context_Map ,
    subsort Context_Map < Entry .    
                
    mb T:term : V:Value  : Context_Map .   
        
    sorts Context_Map+ Context_Map* .
    subsorts Context_Map < Context_Map+ < Context_Map* .
    subsort Context_Map  < Entry .    
    subsort Context_Map+ < Entry+ .    
    subsort Context_Map* < Entry* . 
    
    op _,_ : Context_Map* Context_Map* -> Context_Map* [ditto] .
    op _,_ : Context_Map* Context_Map+ -> Context_Map+ [ditto] .                    

    sort Context_Entry .
    subsort Context_Entry  < Entry .    
    
    vars P : Entry .
    cmb P : Context_Entry if "@context" : { CE*:context_entry+ } := P .  

    ---  Context semantics:    

    vars T   : Term .
    vars Iri : IRI .
    vars V   : Value . 
    vars ... : context_entry* .
        
    eq { "@context" : { T : Iri }, T : V, ...} = { "@context" : { T : Iri }, Iri : V, ...} .

    --- The Identity entry 
    
    sort Identity .
    subsort Identity < Entry .
    cmb P:Entry : Identity if "@id" : Iri:IRI := P .

    --- The Type entry

    sort Type-Entry .
    subsort Type-Entry < Entry .
    cmb P:Entry : Type-Entry if "@type" : Iri:IRI := P .

endfm        

In [ ]:
%%maude 
show modules .

In [ ]:
%%maude 
red "@context" .    
red "pepe" .    
red    

In [ ]:
%%maude 
--- Example 4
red 
 "@context" : {
    "name" : "http://schema.org/name",
    "image" : {
      "@id" : "http://schema.org/image",
      "@type" : "@id"
    }
} .

In [ ]:
%%maude
red {
  "@context" : {
    "name" : "http://schema.org/name"
  },
  "name" : "Manu Sporny",
  "status" : "trollin'"
} .

### HTML

## TO-DO
* Remove trailibg newlines grom cell.
* A command that ads a path to MAUDE_LIB

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()